In [0]:
%pip install -q -U apify-client

In [0]:
from apify_client import ApifyClient
from pyspark.sql.functions import col, current_timestamp, timestamp_diff

In [0]:
# Initialize the ApifyClient with your API token
client = ApifyClient(dbutils.secrets.get("job_notification", "Apify_Token"))
 
# Prepare the Actor input
run_input = {
    "keyword": "Data Engineer",
    "fetchDetails": False,
    "maxJobs": 50,
    "freshness": "1",
    "sortBy": "relevance",
    "experience": "3",
    "cities": ["17","183", "6108", "139"],
}
 
# Run the Actor and wait for it to finish
run = client.actor("alpcnRV9YI9lYVPWk").call(run_input=run_input)
data = client.dataset(run["defaultDatasetId"]).iterate_items()

In [0]:
l = []
for i in data:
    l.append(i)

In [0]:
df = spark.createDataFrame(l)
df = df.withColumn("hrs_ago", timestamp_diff("hour",  col("createdDate").cast("timestamp"), current_timestamp()))
df.write.mode('overwrite').option("mergeSchema", "true").saveAsTable("raw_catalogue.jobs.naukarijobs")
display(df)